# CTE

A common table expression is a named temporary result set that exists only within the execution scope of a single SQL statement e.g.,SELECT, INSERT, UPDATE, or DELETE.

The basic syntax of a CTE is as follows:

```mysql
WITH cte_name (column_list) AS (
    # a query
)
# a query referencing to cte_name
```

(column_list) is optional. The number of columns in column_list should be the same as that in the query, but they don't have the same names. The column names of the cte is those in column_list. If (column_list) is not given, the column names of the query inside the cte are used.


```mysql
WITH ctl AS (
    ...
)
SELECT ... 
FROM tbl INNER JOIN ctl USING (x);
```

Multiple CTEs can be used in the same query.

```mysql
WITH ctl1 AS (
    # a query
),
ctl2 AS (
    # a query referencing to ctl1
) 
SELECT ...
```

## Recursive CTE

A CTE can be recursive:

```mysql
WITH RECURSIVE cte_name AS (
    # initial query called the anchor member
    UNION ALL # or UNION DISTINCT
    # a query referencing to cte_name, called the recursive member
)
# a query referencing to cte_name
```

The recursive member 

* should have a condition stopping the recursion,

* cannot contain aggregate functions, GROUP BY, ORDER BY, and LIMIT, 

* cannot contain DISTINCT if UNION ALL is used,

* can reference the cte only once and in its FROM clause.



The following example uses a recursive CTE generating a row sequentially and unioning all.

```mysql
WITH RECURSIVE cte_count (n) 
AS (
      SELECT 1
      UNION ALL
      SELECT n + 1 
      FROM cte_count 
      WHERE n < 3              # A termination condition is necessary.
    )
SELECT n 
FROM cte_count;

+------+
| n    |
+------+
|    1 |
|    2 |
|    3 |
+------+
```


Assume that we have a table named tree. id and pid (parent id) are among the columns of the table. Each row in the table corresponds to a node in a tree structure. The pid of the root node is NULL. We want to create a query showing id, pid, and depth in ascending order of depth, where depth is the depth of a node (1 for the root node, 2 for the children nodes of the root node, and so on).

Plan:

1. Create the root node: (id, pid=NULL, depth=1)

2. Assume the current cte contains all records with depth=k. To construct the next cte containing all records with depth=k+1, we join the current cte with tree on tree.pid = cte.id

```mysql
WITH RECURSIVE cte AS (
  SELECT id, pid, 1 depth FROM tree WHERE pid IS NULL
  UNION ALL
  SELECT t.id, t.pid, depth+1 FROM tree AS t INNER JOIN cte ON cte.id = t.pid)
SELECT id, pid, depth
FROM cte
ORDER BY depth;
```

An example from https://www.mysqltutorial.org/mysql-recursive-cte/ is as follows:

```mysql
WITH RECURSIVE employee_paths AS (
    SELECT employeeNumber, # id
           reportsTo managerNumber, # pid
           officeCode,   # will be used when we join the cte with another table
           1 lvl    # depth
    FROM employees
    WHERE reportsTo IS NULL
    UNION ALL
    SELECT e.employeeNumber, e.reportsTo, e.officeCode, lvl+1
    FROM employees e INNER JOIN employee_paths ep ON ep.employeeNumber = e.reportsTo)
SELECT employeeNumber, managerNumber, lvl, city
FROM employee_paths ep INNER JOIN offices o USING (officeCode)
ORDER BY lvl, city;
```

# Dynamic SQL

If MySQL server executes a prepared statement, it does not need to fully parse the query. It will simply use the value assigned in the placeholder.

```sql
SET @stmt = 'SELECT * FROM tbl';
PREPARE qry FROM @stmt;
EXECUTE qry;
DEALLOCATE PREPARE qry;
```
In the above example, assume that the table name tbl does not exists. Then we have the error ERROR 1146 (42S02): Table 'currentdb.tbl' doesn't exist. Note that we have the error in the prepare stage, not in the execute stage. If there is an error in the prepare stage, we cannot do the execute and deallocate prepare statements.


```sql
SET @stmt = 'SELECT x, y FROM tbl WHERE x = ?';           # ? is a placeholder.
PREPARE qry FROM @stmt;
SET @x_val = 5; EXECUTE qry USING @x_val;
SET @x_val = 7; EXECUTE qry USING @x_val;
DEALLOCATE PREPARE qry;
```

An example from https://www.mysqltutorial.org/

```sql
SET @schema = 'classicmodels';
SET @pattern = 'o%';
SELECT CONCAT('DROP TABLE ', GROUP_CONCAT(CONCAT(@schema, '.', TABLE_NAME)), ';') INTO @droplike
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_SCHEMA = @schema AND TABLE_NAME LIKE @pattern;

SELECT @droplike;
# DROP TABLE classicmodels.offices,classicmodels.orderdetails,classicmodels.orders;

PREPARE stmt FROM @droplike;
EXECUTE stmt;
DEALLOCATE PREPARE stmt;
```

# Stored programs

Four types of stored programs:

1. Stored function: called from a SQL statement

1. Stored procedure: called from an application that has access to the database

1. Trigger: executed when INSERT, UPDATE, or DELETE is applied on a table

1. Event: executed at a scheduled time


## DECLARE, SET, CURSOR


```sql
DECLARE x DECIMAL(10,2) DEFAULT 3.14;
DECLARE y INT;

SET y = 10;
SELECT MAX(amount) INTO x FROM tbl WHERE id = z;
```

SQL statements work with an entire result set by default. If you need to work each row in the result set at a time, use `CURSOR`. Note that the standard database access is faster than using `CURSOR`.

```sql
DECLARE row_not_found TINYINT DEFAULT FALSE;
DECLARE CONTINUE HANDLER FOR NOT FOUND SET row_not_found = TRUE;

DECLARE cursor_name CURSOR FOR 
SELECT ... FROM tbl WHERE ...;

OPEN cursor_name;

WHILE row_not_found = FALSE DO
  FETCH cursor_name INTO variable(s);
END WHILE;

CLOSE cursor_name;
```

### DECLARE {CONTINUE | EXIT} HANDLER FOR condition

condition:

* `NOT FOUND`
* `SQLEXCEPTION`
* `SQLWARNING`
* error_code or `SQLSTATE` sqlstate_code

```sql
DECLARE CONTINUE HANDLER FOR 1329 SET not_exists = TRUE

DECLARE CONTINUE HANDLER FOR SQLSTATE '02000' SET not_exists = TRUE

DECLARE CONTINUE HANDLER FOR NOT FOUND SET not_exists = TRUE

IF not_exists = TRUE THEN 
  ...
ELSE
  ...
END IF;
```

Example of EXIT:

```sql
CREATE PROCEDURE my_proc()
BEGIN
  DECLARE duplicate_entry TINYINT DEFAULT FALSE;
  DECLARE null_column TINYINT DEFAULT FALSE;
  DECLARE sql_exception TINYINT DEFAULT FALSE;
  
  BEGIN
    DECLARE EXIT HANDLER FOR 1062 SET duplicate_entry = TRUE;
    DECLARE EXIT HANDLER FOR 1048 SET null_column = TRUE;
    DECLARE EXIT HANDLER FOR SQLEXCEPTION SET sql_exception = TRUE;
    
    INSERT INTO tbl VALUES (10, 'Man');                      
    SELECT '1 row inserted' AS result;
  END;
  IF duplicate_entry = TRUE THEN 
    SELECT 'Cannot insert a duplicate entry' AS result;
  ELSEIF null_column = TRUE THEN
    SELECT 'Column should be null' AS result;
  ELSEIF sql_exception = TRUE THEN
    SELECT 'SQL exception' AS result;
  END IF;
END
```

## Stored functions

A stored function is also called a user-define function (UDF) or just a function. With MySQL, you can only create scalar functions.

Example:

```sql
CREATE FUNCTION city_pop(city_name CHAR(35), country_code CHAR(3))
RETURNS INT READS SQL DATA
BEGIN
  DECLARE pop INT;
  DECLARE CONTINUE HANDLER FOR NOT FOUND SET pop = 0;
  SELECT Population INTO pop FROM world.city 
  WHERE CountryCode = country_code AND Name = city_name;
  RETURN pop;
END

SELECT city_pop('Chicago', 'USA');
```


### Inline table-valued functions (TVF)

SQL Server: 

```sql
CREATE FUNCTION func(@x AS INT) RETURNS TABLE
AS
RETURN
  SELECT ...
  FROM tbl
  WHERE id = @x;
  
SELECT ...
FROM func(2) AS t;

SELECT ...
FROM func(3) AS tbl1 INNER JOIN tbl2 ON ...;
```

## Procedure

### MySQL

Example:

```sql
DELIMITER //

CREATE PROCEDURE my_proc()
BEGIN
  DECLARE sum_x DECIMAL(10, 3);
  
  SELECT SUM(x) INTO sum_x
  FROM tbl WHERE id = 50;
  
  IF sum_x > 10 THEN
    SELECT CONCAT('Sum of x when id = 50: ', sum_x) AS sum_x_id50;
  ELSE
    SELECT 'Insufficient' AS sum_x_id50;
  END IF;
END//

DELIMITER ;
CALL my_proc();
```

Example:

```sql
DELIMITER //
CREATE PROCEDURE my_proc(x INT, y DECIMAL(10,2))
BEGIN
  DECLARE sql_error TINYINT DEFAULT FALSE;
  DECLARE CONTINUE HANDLER FOR SQLEXCEPTION SET sql_error = TRUE;
  
  IF y IS NULL THEN 
    SET y = 1;
  END IF;
  
  START TRANSACTION;
  UPDATE ...;
  IF sql_error = FALSE THEN
    COMMIT;
  ELSE
    ROLLBACK;
  END IF;
END //
```

Example:

```sql
DELIMITER //

CREATE PROCEDURE max_city_pop(IN country_code CHAR(3), OUT city_name CHAR(35), OUT pop INT)
BEGIN
  SELECT IFNULL(MAX(Population),0) INTO pop FROM world.city 
  WHERE CountryCode = country_code;
  
  SELECT IF(MAX(Name) IS NULL, 'Unknown', Name) INTO city_name FROM world.city 
  WHERE CountryCode = country_code AND Population = pop;
END

DELIMITER ;

CALL max_city_pop('KOR', @cityname, @maxpop);
SELECT @cityname, @maxpop;
```

Example from https://www.mysqltutorial.org/

```sql
DELIMITER //

CREATE PROCEDURE check_table_exists(IN table_name VARCHAR(100), OUT table_exists TINYINT(1))
BEGIN
    DECLARE CONTINUE HANDLER FOR 1146 SET @err = 1;
    SET @err = 0;
    SET @table_name = table_name;
    SET @query = CONCAT('SELECT 1 FROM ', @table_name, ';');
    PREPARE stmt FROM @query;
    SELECT IF(@err = 1, 0, 1) INTO table_exists;
    IF(@err=0) THEN
        DEALLOCATE PREPARE stmt;
    END IF;
END //

CALL check_table_exists('my_tbl', @table_exists);
SELECT @table_exists;
```

Note that in the above example we don't use an EXECUTE statement. It is because we will have an error in the PREPARE stage when table_name does not exist.

### SQL Server

```sql
DROP PROC IF EXISTS proc_name;
GO

CREATE PROC proc_name
  @x AS NVARCHAR(50)
AS
SELECT ...
FROM ...
WHERE col = @x;
GO

EXEC proc_name @x=10;
```

## Triggers

Triggers are commonly used to enforce data consistency.


### CREATE TRIGGER

```sql
CREATE TRIGGER trigger_name {BEFORE|AFTER} {INSERT|UPDATE|DELETE}
ON tbl_name FOR EACH ROW
-- trigger statements including NEW and OLD 
-- (enclosed by BEGIN and END if there exists multiple statements).
```

* The NEW table is used in INSERT and UPDATE triggers.

* The OLD table is used in DELETE and UPDATE triggers.

Example:

```sql
DELIMITER //

CREATE TRIGGER trig_before_update
  BEFORE UPDATE ON tbl
  FOR EACH ROW
BEGIN
  SET NEW.col = UPPER(NEW.col);
END//

DELIMITER ;

UPDATE tbl SET col = 'apple' WHERE id = 10;
```

Example:

```sql
CREATE TRIGGER tbl_before_update
  BEFORE UPDATE ON tbl
  FOR EACH ROW
BEGIN
  DECLARE x DECIMAL(10,2);
  
  SELECT ... INTO x FROM ... WHERE id = NEW.id;
  
  IF x != NEW.y THEN
    SIGNAL SQLSTATE 'HY000' SET MESSAGE_TEXT = 'Mismatch.';
  END IF;
END
```

### Show triggers

```sql
SHOW TRIGGERS;

SHOW TRIGGERS IN database_name;

SHOW TRIGGERS IN database_name LIKE 'test%';

SHOW TRIGGERS FROM db_name WHERE `table` = 'tbl_name';

SELECT TRIGGER_NAME, EVENT_OBJECT_TABLE, EVENT_MANIPULATION
FROM INFORMATION_SCHEMA.TRIGGERS
WHERE TRIGGER_SCHEMA = 'db_name' 
ORDER BY ACTION_TIMING; 
```

### Multiple triggers

Use FOLLOWS or PRECEDES.

```sql
CREATE TRIGGER new_trigger
{BEFORE|AFTER} {INSERT|UPDATE|DELETE}
ON tbl_name FOR EACH ROW
{FOLLOWS|PRECEDES} old_trigger
-- trigger body
```

### Calling a procedure in a trigger

A trigger cannot call a stored procedure if the procedure has OUT or INOUT parameters or uses dynamic SQL.


This is an emulator for CHECK (https://www.mysqltutorial.org):

```sql
DELIMITER //
CREATE PROCEDURE check_parts(IN cost DECIMAL(10,2), IN price DECIMAL(10,2))
BEGIN
  IF cost < 0 THEN
      SIGNAL SQLSTATE '45000' SET MESSAGE_TEXT = 'check constraint on parts.cost failed';
  END IF;
  
  IF price < 0 THEN
      SIGNAL SQLSTATE '45001' SET MESSAGE_TEXT = 'check constraint on parts.price failed';
  END IF;
  
  IF price < cost THEN
      SIGNAL SQLSTATE '45002' SET MESSAGE_TEXT = 'check constraint on parts.price and parts.cost failed';
  END IF;
END//
DELIMITER ;

DELIMITER //
CREATE TRIGGER parts_before_insert BEFORE INSERT ON parts
FOR EACH ROW
BEGIN
    CALL check_parts(new.cost, new.price);
END//
DELIMITER ;

DELIMITER //
CREATE TRIGGER parts_before_update BEFORE UPDATE ON parts
FOR EACH ROW
BEGIN
    CALL check_parts(new.cost, new.price);
END//
DELIMITER ;
```

# SIGNAL


In a stored program, we can do _data validation_ by using SIGNAL. A SIGNAL statement raises an error.

```sql
IF ... THEN
    SIGNAL SQLSTATE 'state_value' SET MYSQL_ERRNO = err_no, MESSAGE_TEXT = 'msg';
END IF;
```

Example

```sql
DELIMITER //

CREATE PROCEDURE my_proc(x INT, y DECIMAL(10,2))
BEGIN
  -- data validation of y
  IF y < 0 THEN
    SIGNAL SQLSTATE '22003' SET MESSAGE_TEXT = 'y cannot be negative.', MYSQL_ERRNO = 1264;
  ELSEIF y > 100 THEN
    SIGNAL SQLSTATE '22003' SET MESSAGE_TEXT = 'y cannot be greater than 100.', MYSQL_ERRNO = 1264;
  END IF;
  
  UPDATE ...;
END //
```

# Transaction, Locking


## Transaction

```sql
START TRANSACTION
-- execute statements
COMMIT; -- or ROLLBACK;
```
In the above example, assume that we do a DELETE clause on a table between the two lines START TRANSACTION and COMMIT. If we see the table from another session, we can still see the original table. After COMMIT is done, the data will be deleted permanently. If we use ROLLBACK, the rows deleted will be placed back into the table.


```sql
START TRANSACTION
-- execute statements
SAVEPOINT save_pt;
-- execute statements
ROLLBACK TO SAVEPOINT save_pt;
COMMIT;

SET autocommit = 0;        -- or SET autocommit = OFF;
-- execute statements
COMMIT;   -- or ROLLBACK
SET autocommit = 1;        -- or SET autocommit = ON;
```

Example:

```sql
CREATE PROCEDURE my_proc()
BEGIN
  DECLARE sql_exception TINYINT DEFAULT FALSE;
  DECLARE CONTINUE HANDLER FOR SQLEXCEPTION SET sql_error = TRUE;
  
  START TRANSACTION;
  
  INSERT INTO tbl VALUES ...;
  INSERT INTO tbl VALUES ...;
  
  IF sql_error = FALSE THEN
    COMMIT;
    SELECT 'The transaction was done successfully.';
  ELSE
    ROLLBACK;
    SELECT 'The transaction failed.';
  END IF;
END
```


## LOCK


```sql
LOCK TABLES tbl [READ | WRITE];

LOCK TABLES tbl1 [READ | WRITE],  tbl2 [READ | WRITE];

UNLOCK TABLES;
```

If the READ lock is acquired in the session 1, we cannot write data to the table in the session 1. If we try to write data to the table in the session 2, then the write operations will be put into the waiting states until the read lock is released in the session 1.


The session information about waiting states can be shown by 

```sql
SHOW PROCESSLIST;
```

### SET TRANSACTION ISOLATION LEVEL

A way to prevent concurrency problems (lost updates, dirty reads, nonrepeatable reads, phantom reads) is to change the default locking behavior. 

```sql
SET [GLOBAL | SESSION] TRANSACTION ISOLATION LEVEL 
    {READ UNCOMMITTED | READ COMMITTED | REPEATABLE READ | SERIALIZABLE}
```

* `READ UNCOMMITTED` allows all concurrency problems.
* `READ COMMITTED` prevents only dirty reads.
* `REPEATABLE READ` allows only phantom reads.
* `SERIALIZABLE` prevents all concurrent problems.

### deadlocks

A deadlock occurs when neither of two transactions can be committed.

# Event

By default, the event scheduler is off. 

```sql
SHOW VARIABLES LIKE 'event_scheduler';

SET GLOBAL event_scheduler = ON;

CREATE EVENT event_name
ON SCHEDULE AT NOW() + INTERVAL 1 MONTH
-- ON SCHEDULE EVERY 1 MONTH STARTS '2000-01-01'
DO BEGIN
  ...
END
```

To show, alter, and drop events:

```sql
SHOW EVENTS;
SHOW EVENTS IN database_name;
SHOW EVENTS IN database_name LIKE 'test%';

ALTER EVENT event_name DISABLE;
ALTER EVENT event_name ENABLE;
ALTER EVENT event_name RENAME TO evt;

DROP EVENT [IF EXISTS] event_name;
```